If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers, 🤗 Datasets and 🤗 Accelerate. Uncomment the following cell and run it.

In [1]:
%pip install wandb transformers datasets evaluate accelerate --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 252.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 294.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 254.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.9/801.9 kB 342.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 301.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 129.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 235.9 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9
Note: you may need to restart the kernel to use updated packages.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!conda install -c conda-forge git-lfs -y

Channels:
 - conda-forge
Platform: linux-64
doneecting package metadata (repodata.json): - 
doneing environment: | 


==> WARNING: A newer version of conda exists. <==
    current version: 25.1.1
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - git-lfs


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.8.3   |       hbd8a1cb_0         151 KB  conda-forge
    certifi-2025.8.3           |     pyhd8ed1ab_0         155 KB  conda-forge
    git-lfs-3.7.0              |       h59e48b9_0         4.2 MB  conda-forge
    openssl-3.5.2              |       h26f9b46_0         3.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         7.5 MB

The follo

# Fine-tuning a model on a text classification task

In [4]:
task = "nli"
model_checkpoint = "distilbert/distilbert-base-multilingual-cased"
batch_size = 32

## Loading the dataset

In [5]:
from datasets import load_dataset, Dataset, DatasetDict

In [22]:
import pandas as pd

df_train = pd.read_csv("nli_train.csv", nrows=None,  # Don't limit rows
                 low_memory=False).reset_index(drop=True)
df_val = pd.read_csv("nli_val.csv")
df_test = pd.read_csv("nli_test.csv")

In [7]:
df_train

,sentence,l1
0,"Z drugimi besedami, nimamo spoštovanja do indu...",Albania
1,"Dinamično življenje, pomanjkanje časa in podal...",Albania
2,Nato pa moram to posodo sama oprati ali jo sam...,Albania
3,Ve Čtverteu za 9:00 - 13:00 hodin studuju češt...,Albania
4,Zaradi dobre korporacijske organiziranost in š...,Albania
...,...,...
518155,"Drug users shouldn’t be arrested, they should ...",Wales
518156,Drivers were not insane maniacs,Wales
518157,Meaning 'And his blood was mixed with the rain,Wales
518158,And yet you still use the English language for...,Wales


In [23]:
import evaluate
metric = evaluate.load("accuracy")

dataset = DatasetDict()
dataset['train'] = Dataset.from_pandas(df_train)
dataset['validation'] = Dataset.from_pandas(df_val)
dataset['test'] = Dataset.from_pandas(df_test)

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'l1'],
        num_rows: 485982
    })
    validation: Dataset({
        features: ['sentence', 'l1'],
        num_rows: 161994
    })
    test: Dataset({
        features: ['sentence', 'l1'],
        num_rows: 161994
    })
})

To access an actual element, you need to select a split first, then give an index:

In [10]:
dataset["train"][0]

{'sentence': 'Z drugimi besedami, nimamo spoštovanja do industrije hrane oziroma nimamo pozitivnega odnosa do nje.',
 'l1': 'Albania'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [11]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [12]:
show_random_elements(dataset["train"])

,sentence,l1
0,se je učila do dvanajst zvečer.,North Macedonia
1,"Jsem moc rada, že mam takovou kamarádku.",Russia
2,See your coastline there is a little shorter than Italia's,United Kingdom
3,"Za božic smo šli na vas kod moje babice, kje gremo vsako leto.",Serbia
4,Popoldne vsak dan kuha hitro kosilo za družino.,Argentina
5,"She is not a teporter, she is one of the (rotestors (and the Swcretary GeneEal of the rerman libe$als )",Bavaria
6,"Azerbaijani : About 95%, almost native, sounds like a Turkish dialect",Turkey
7,I have seen nature's fiercest predators tamed and giving hugs to their owners who fed them from day one,Turkey
8,The US paGs 22% or ~ $ 365 Hillion,Greenland
9,Nedele - brzo volno a v 15:00 jsem hraju v tenis.,Russia


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [25]:
unique_labels = list(set(list(df_train.l1.unique()) + list(df_test.l1.unique()) + list(df_val.l1.unique())))
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

def preprocess_function(examples):
    labels = [label2id[label] for label in examples["l1"]]
    tokenized = tokenizer(examples["sentence"], truncation=True, max_length=512)
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "labels": labels
    }

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [26]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/485982 [00:00<?, ? examples/s]

Map:   0%|          | 0/161994 [00:00<?, ? examples/s]

Map:   0%|          | 0/161994 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

In [27]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'l1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 485982
    })
    validation: Dataset({
        features: ['sentence', 'l1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 161994
    })
    test: Dataset({
        features: ['sentence', 'l1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 161994
    })
})

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always 2, except for STS-B which is a regression problem and MNLI where we have 3 labels):

In [52]:
from transformers import DistilBertForSequenceClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = DistilBertForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
    )
model.config.hidden_dropout_prob = 0.5
model.config.attention_probs_dropout_prob = 0.5

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [53]:
metric_name = "accuracy"
# model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    "benali",
    eval_strategy = "epoch",
    save_strategy = "no",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size * 2,
    per_device_eval_batch_size=batch_size * 4,
    num_train_epochs=4,
    weight_decay=0.01,
    metric_for_best_model=metric_name,
    push_to_hub=True,
    save_total_limit=2,
    save_only_model=True,
    dataloader_pin_memory=False,
    remove_unused_columns=True
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-mrpc"` or `"huggingface/bert-finetuned-mrpc"`).

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [54]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [55]:
from transformers import DataCollatorWithPadding
from transformers import EarlyStoppingCallback

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

validation_key = "validation"
# encoded_dataset = encoded_dataset.remove_columns(dataset["train"].column_names)
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [56]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
trainer.train()

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


Epoch,Training Loss,Validation Loss,Accuracy
1,2.623500,2.612017,0.323765
2,2.472100,2.556456,0.334068
3,2.320800,2.566949,0.336932
4,2.150800,2.616009,0.334080


TrainOutput(global_step=30376, training_loss=2.425156298705958, metrics={'train_runtime': 2261.9908, 'train_samples_per_second': 859.388, 'train_steps_per_second': 13.429, 'total_flos': 3.942255226363344e+16, 'train_loss': 2.425156298705958, 'epoch': 4.0})

In [57]:
predictions = trainer.predict(encoded_dataset["test"])

In [58]:
predictions

PredictionOutput(predictions=array([[ -1.1306971 ,  -2.6161263 ,  -6.341968  , ...,   1.1613284 ,
         -2.3883183 , -13.948739  ],
       [ -5.8590326 ,  -2.8533332 ,  -4.7756486 , ...,  -4.9771924 ,
         -3.4311776 ,  -2.1962695 ],
       [  0.3818953 ,  -0.25668234,  -3.9851155 , ...,  -1.0996433 ,
         -0.5731757 , -10.420356  ],
       ...,
       [ -0.23648402,  -0.04409911,  -4.0522017 , ...,  -0.06994177,
         -0.5446305 , -10.515467  ],
       [  1.2080548 ,   0.76065385,  -4.757542  , ...,   0.5169087 ,
         -1.3770596 , -10.595108  ],
       [ -3.3310735 ,  -1.4940068 ,  -5.1123796 , ...,  -1.7960906 ,
         -2.6751657 ,  -7.583517  ]], dtype=float32), label_ids=array([17, 43, 31, ..., 76, 33, 40]), metrics={'test_loss': 2.607670307159424, 'test_accuracy': 0.3362099830857933, 'test_runtime': 58.413, 'test_samples_per_second': 2773.255, 'test_steps_per_second': 21.673})

In [59]:
pred_logits = predictions.predictions
true_labels = predictions.label_ids
pred_class_ids = np.argmax(pred_logits, axis=1)

In [60]:
from scipy.special import softmax

y_pred = []
y_true = []

for i in range(len(pred_class_ids)):
    pred_label = id2label[pred_class_ids[i]]
    true_label = id2label[true_labels[i]] if true_labels is not None else "N/A"
    y_pred.append(pred_label)
    y_true.append(true_label)


In [61]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                        precision    recall  f1-score   support

               Albania       0.57      0.12      0.20       251
               Andorra       0.00      0.00      0.00        15
             Argentina       0.48      0.04      0.08       296
               Armenia       0.55      0.06      0.11        97
             Astrakhan       0.33      0.02      0.04        52
             Australia       0.43      0.12      0.19       602
               Austria       0.38      0.18      0.24      3672
            Azerbaijan       0.00      0.00      0.00        44
               Bavaria       0.00      0.00      0.00       214
               Belarus       0.25      0.07      0.11       213
               Belgium       0.46      0.19      0.27      3219
Bosnia and Herzegovina       0.38      0.19      0.25      3613
                Brazil       0.73      0.08      0.14       144
              Bulgaria       0.38      0.19      0.25      1449
              Cambodia       0.00      

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

In [47]:
del trainer